In [ ]:
!pip install -q pyarrow

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm 
import scipy
from datetime import timedelta

# magiya

In [ ]:
df = pd.read_csv("emj_df4.csv")
pst_df = pd.read_parquet("player_starts_train.parquet", engine='pyarrow')

In [ ]:
df

In [ ]:
pst_df

In [ ]:
min_date = pst_df['date'].min()
max_date = pst_df['date'].max()

print("Минимальная дата:", min_date)
print("Максимальная дата:", max_date)
max_date = pd.Timestamp(max_date)

In [ ]:
selected_dates = []
selected_dates.append(max_date)
for _ in range(4):
    max_date -= timedelta(days=7)
    selected_dates.append(max_date)
selected_dates.reverse()

print("Выбранные даты для разделения месяца на недели:")
for date in selected_dates:
    print(date)

In [ ]:
dividing_dates = [
    pd.Timestamp('2023-08-21 23:59:59+0300', tz='UTC+03:00')
]

In [ ]:
def calculate_is_watched(row):
    user_id = row['user_id']
    item_id = row['item_id']
    is_watched = row['is_watched']

    matching_row = pst_df[(pst_df['user_id'] == user_id) & (pst_df['item_id'] == item_id)]
    if not matching_row.empty:
        date_str = matching_row.iloc[0]['date'] 
        date = pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S%z', errors='coerce')  

        for dividing_date in dividing_dates:
            if pd.notna(date) and date <= dividing_date:
                deltat = max((dividing_date - date).total_seconds(), 0)
                r0 = is_watched
                r = r0 * (1 - 0.25 * (deltat * 1000 / 604800000))
                return r
    return is_watched

In [ ]:
df['is_watched'] = df.apply(lambda row: calculate_is_watched(row), axis=1)